In [2]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image


In [10]:

nutrition_df = pd.read_csv("indian_food_nutrition.csv")
nutrition_df.head()


,food,calories,fat,protein,carbs
0,adhirasam,250.0,8.0,3.0,45.0
1,aloo_gobi,150.0,8.0,4.0,20.0
2,aloo_matar,180.0,9.0,5.0,22.0
3,aloo_methi,160.0,7.0,4.0,20.0
4,aloo_shimla_mirch,170.0,8.0,4.0,22.0


In [11]:
train_dir = "D:\Data science\datasets\Indian Food Images\Indian Food Images"


datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 5814 images belonging to 90 classes.


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\harsh\AppData\Local\Temp\ipykernel_30672\4279575907.py:1: SyntaxWarning: invalid escape sequence '\D'
  train_dir = "D:\Data science\datasets\Indian Food Images\Indian Food Images"


Found 1449 images belonging to 90 classes.


In [14]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


In [15]:
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10  
)


In [ ]:
model.save("indian_food_model.h5")


In [8]:
# Load and preprocess new image
img_path = "ras.jpeg"
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0
model = load_model("indian_food_model.h5")
# Predict
pred = model.predict(img_array)
class_index = np.argmax(pred)
food_class = list(train_generator.class_indices.keys())[class_index]

print(f"Predicted Food: {food_class}")


NameError: name 'load_model' is not defined

In [17]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model("indian_food_model.h5")

class_indices = {
    'aloo_gobi': 0,
    'paneer_butter_masala': 1,
    'chole': 2,
    'dal_makhani': 3,
    'idli': 4
}
index_to_class = {v: k for k, v in class_indices.items()}

img_path = "ras.jpeg"
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

pred = model.predict(img_array)
class_index = np.argmax(pred)
food_class = index_to_class[class_index]

print(f"Predicted Food: {food_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


KeyError: 27